# Lista 1

## Imports

In [4]:
import re

## Questão 1

In [1]:
def validade_pass(password: str):
    if len(password) < 8:
        return False
    have_lowercase = False
    have_uppercase = False
    have_number = False
    have_special = False
    for character in password:
        if 'a' <= character <= 'z':
            have_lowercase = True
        elif 'A' <= character <= 'Z':
            have_uppercase = True
        elif '0' <= character <= '9':
            have_number = True
        else:
            have_special = True
    
    is_valid = have_lowercase and have_uppercase and have_number and have_special
    return is_valid

In [3]:
print(validade_pass('123dsad23132'))
print(validade_pass('123dsad23132#'))
print(validade_pass('12A3d2#'))
print(validade_pass('123dA2#'))
print(validade_pass('123dAaa2#'))

False
False
False
False
True


## Questão 2

## Questão 3

In [249]:
def get_metadata(apa_citation: str):
    authors = re.findall(r"[A-Z][a-z]+, [A-Z]\.(?: [A-Z]\.)?", apa_citation)
    year = re.search(r"([0-9]+)", apa_citation).group(0)
    title = re.search(r"(?<=\)\. )[A-Za-z :]+", apa_citation).group(0)
    publisher = re.search(r"(?<= )[A-Za-z :]+\.$", apa_citation).group(0)
    return authors, year, title, publisher

In [250]:
test_str = 'Manning, C. D., Manning, C. D., & Schutze, H. (1999). Foundations: of statistical natural language processing. MIT press.'
authors, year, title, publisher = get_metadata(test_str)
print(authors)
print(year)
print(title)
print(publisher)

['Manning, C. D.', 'Manning, C. D.', 'Schutze, H.']
1999
Foundations: of statistical natural language processing
MIT press.


## Questão 4

## Questão 5

## Questão 6

## Questão 7